# Neural Network Algorithm using TensorFlow

Neural Network is, a directed graph, organized by layers and layers are created by number of interconnected neurons (or nodes). Every neuron in a layer is connected with all the neurons from previous layer; there will be no interaction of neurons within a layer. The performance of a Neural Network is measured using cost or error function and the dependent input weight variables. Forward-propagation and back-propagation are two techniques, neural network uses repeatedly until all the input variables are adjusted or calibrated to predict accurate output. During, forward-propagation, information moves in forward direction and passes through all the layers by applying certain weights to the input parameters. Back-propagation method minimizes the error in the weights by applying an algorithm called gradient descent at each iteration step. We have used TensorFlow python library to predict the sale price of housing dataset using simple feed-forward neural network. TensorFlow uses tensors, special multi-dimensional arrays to store the datasets for easier linear algebra and vector calculus operations.

In [48]:
import os
import pandas as pd
import numpy as np 
from sklearn.metrics import mean_squared_error
import tensorflow as tf
import matplotlib.pyplot as plt

%matplotlib inline

In [49]:
print(os.getcwd())
print("")

train = pd.read_csv("../data/train_after_feature_engineering.csv")
test = pd.read_csv("../data/test_after_feature_engineering.csv")

print ('The train data has {0} rows and {1} columns'.format(train.shape[0],train.shape[1]))    
print ('The test data has {0} rows and {1} columns'.format(test.shape[0],test.shape[1]))


/home/mcheruvu/notebook/code

The train data has 1460 rows and 307 columns
The test data has 1459 rows and 306 columns


In [50]:
np.random.seed(1234)

train["SalePrice"] = np.log1p(train["SalePrice"]) # log(SalePrice) + 1

# Shuffle data
train = train.sample(frac=1)

# Split into training, cross validation, and testing datasets
train, cv1, cv2 = np.split(train,
                        [int(.6 * len(train)), 
                         int(.8 * len(train))])

# Convert into numpy arrays
x_train = train.drop(['SalePrice', 'Id'], axis=1).as_matrix().astype(np.float32)
y_train = train['SalePrice'].as_matrix().astype(np.float32).reshape((np.shape(x_train)[0], 1))

x_cv1 = cv1.drop(['SalePrice', 'Id'], axis=1).as_matrix().astype(np.float32)
y_cv1 = cv1['SalePrice'].as_matrix().astype(np.float32).reshape((np.shape(cv1)[0], 1))

x_cv2 = cv2.drop(['SalePrice', 'Id'], axis=1).as_matrix().astype(np.float32)
y_cv2 = cv2['SalePrice'].as_matrix().astype(np.float32).reshape((np.shape(cv2)[0], 1))


# Fit the Model

In [51]:
train_size = np.shape(x_train)[0]
cv1_size = np.shape(x_cv1)[0]
cv2_size = np.shape(x_cv2)[0]

num_features = np.shape(x_train)[1]
num_hidden = 16 # Number of activation units in the hidden layer

graph = tf.Graph()
with graph.as_default():
    
    # Input
    tf_train_dataset = tf.constant(x_train, dtype=tf.float32)
    tf_train_labels = tf.constant(y_train, dtype=tf.float32)
    
    tf_valid_dataset = tf.constant(x_cv1)
    tf_test_dataset = tf.constant(x_cv2)
    
    # Variables
    weights_1 = tf.Variable(tf.truncated_normal(
        [num_features, num_hidden]), dtype=tf.float32, name="layer1_weights")
    
    biases_1 = tf.Variable(tf.zeros([num_hidden]), dtype=tf.float32, name="layer1_biases")
    
    weights_2 = tf.Variable(tf.truncated_normal(
        [num_hidden, 1]), dtype = tf.float32, name="layer2_weights")
    
    biases_2 = tf.Variable(tf.zeros([1]), dtype=tf.float32, name="layer2_biases")
    
    steps = tf.Variable(0)
    
    # Model
    def model(x, train=False):
        hidden = tf.nn.relu(tf.matmul(x, weights_1) + biases_1)
        return tf.matmul(hidden, weights_2) + biases_2
    #end def
    
    # Loss Computation
    train_prediction = model(tf_train_dataset)
    loss = 0.5 * tf.reduce_mean(tf.squared_difference(tf_train_labels, train_prediction))
    cost = tf.sqrt(loss)
    
    # Optimizer
    # Exponential decay of learning rate
    learning_rate = tf.train.exponential_decay(0.06, steps, 5000, 0.70, staircase=True)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost, global_step=steps)
    
    # Predictions
    valid_prediction = model(tf_valid_dataset)
    test_prediction = model(tf_test_dataset)
    
    saver = tf.train.Saver()
#end with

# Predict the Test Sale Price

In [52]:
num_steps = 100001

def accuracy(prediction, labels):
    return 0.5 * np.sqrt(((prediction - labels) ** 2).mean(axis=None))
#end def

with tf.Session(graph=graph) as sess:
    
    tf.global_variables_initializer().run()
    
    print('Initialized')
    
    for step in range(num_steps):
        # Run the computations. We tell .run() that we want to run the optimizer,
        # and get the loss value and the training predictions returned as numpy
        # arrays.
        _, c, predictions = sess.run([optimizer, cost, train_prediction])
        if (step % 5000 == 0):
            print('Cost at step %d: %.2f' % (step, c))
            # Calling .eval() on valid_prediction is basically like calling run(), but
            # just to get that one numpy array. Note that it recomputes all its graph
            # dependencies.
            print('Validation loss: %.2f' % accuracy(valid_prediction.eval(), y_cv1))
        #end if
    #end for
    
    t_pred = test_prediction.eval()
    
    print('Test loss: %.2f' % accuracy(t_pred, y_cv2))    
     
    print("Making predictions...")
    
    x = test.drop('Id', axis=1).as_matrix().astype(dtype=np.float32)
    
    hidden = tf.nn.relu(tf.matmul(x, weights_1) + biases_1)
    
    y = tf.cast((tf.matmul(hidden, weights_2) + biases_2), dtype=tf.uint16).eval()
    
    test['SalePrice'] = y
    
    output = test[['Id', 'SalePrice']]    
    
#end with


Initialized
Cost at step 0: 10791.43
Validation loss: 4781325.50
Cost at step 5000: 0.27
Validation loss: 0.20
Cost at step 10000: 0.27
Validation loss: 0.20
Cost at step 15000: 0.27
Validation loss: 0.20
Cost at step 20000: 0.27
Validation loss: 0.20
Cost at step 25000: 0.27
Validation loss: 0.20
Cost at step 30000: 0.27
Validation loss: 0.20
Cost at step 35000: 0.27
Validation loss: 0.20
Cost at step 40000: 0.27
Validation loss: 0.20
Cost at step 45000: 0.27
Validation loss: 0.20
Cost at step 50000: 0.27
Validation loss: 0.20
Cost at step 55000: 0.27
Validation loss: 0.20
Cost at step 60000: 0.27
Validation loss: 0.20
Cost at step 65000: 0.27
Validation loss: 0.20
Cost at step 70000: 0.27
Validation loss: 0.20
Cost at step 75000: 0.27
Validation loss: 0.20
Cost at step 80000: 0.27
Validation loss: 0.20
Cost at step 85000: 0.27
Validation loss: 0.20
Cost at step 90000: 0.27
Validation loss: 0.20
Cost at step 95000: 0.27
Validation loss: 0.20
Cost at step 100000: 0.27
Validation loss: 

# Save Predictions

In [53]:

#df_predict = pd.DataFrame({'Id': test["Id"], 'SalePrice': np.exp(test["SalePrice"]) - 1.0})
#df_predict = pd.DataFrame({'Id': id_vector, 'SalePrice': sale_price_vector})

output.to_csv('../data/kaggle_python_neural_network.csv', header=True, index=False)

print("...file saved")

...file saved
